## Homework: Deep Jazz

In [1]:
import numpy as np

from grammar import unparse_grammar
from music21 import stream, midi, tempo, note
from preprocess import get_musical_data, get_corpus_data
from qa import prune_grammar, prune_notes, clean_up_notes
from generator import __sample, __generate_grammar, __predict

In [2]:
N_epochs = 128  # default
data_fn = 'midi/' + 'original_metheny.mid'  # 'And Then I Knew' by Pat Metheny
out_fn = 'midi/' 'deepjazz_on_metheny...' + str(N_epochs)

In [3]:
%%time

max_len = 20
max_tries = 1000
diversity = 0.5

# musical settings
bpm = 130

# get data
chords, abstract_grammars = get_musical_data(data_fn)
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78
CPU times: user 8.32 s, sys: 177 ms, total: 8.5 s
Wall time: 8.7 s


In [4]:
chords[0]

[<music21.instrument.Piano Piano>,
 <music21.tempo.MetronomeMark Quarter=112.0>,
 <music21.key.Key of G major>,
 <music21.meter.TimeSignature 4/4>]

In [5]:
%%capture
# NOTE: not working anymore

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

def get_keras_model(max_len, N_values):
    # build a 2 stacked LSTM
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(N_values))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

## Task

Replace previous model with equivalent in prettytensor or tf.slim

Try to make you code as compact as possible

In [6]:
import tqdm
import random
import tensorflow as tf
import tensorflow.contrib.slim as slim

from lazy import lazy
from tensorflow.contrib import rnn


def split_batches(n, n_batches):
    indicies = np.arange(n)
    np.random.shuffle(indicies)
    for i in range(len(indicies) // n_batches):
        yield indicies[i * n_batches: (i + 1) * n_batches]


class MyModel:
    def __init__(self, max_len, N_values, lat_size=128, dr_rate=0.2, lr=1e-3):
        self.max_len = max_len
        self.N_values = N_values
        self.lat_size = lat_size
        self.dr_rate = dr_rate
        self.lr = lr
        
        self._make_net()
    
    def fit(self, X, y, batch_size, n_epochs):
        optimizer = tf.train.RMSPropOptimizer(learning_rate=self.lr).minimize(self.loss)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for _ in tqdm.tqdm_notebook(range(n_epochs), desc='Epoch'):
                for inds in split_batches(len(X), batch_size):
                    sess.run(optimizer, feed_dict={self.X: X[inds], self.y: y[inds], self.dr: self.dr_rate})
    
    def predict(self, X):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(self.pred_labels, feed_dict={self.X: X, self.dr: 0.0})
    
    def _make_net(self):
        tf.reset_default_graph()
        
        # Inputs
        self.X = tf.placeholder(tf.float32, shape=[None, self.max_len, self.N_values])
        self.y = tf.placeholder(tf.float32, shape=[None, self.N_values])
        self.dr = tf.placeholder(tf.float32)
        
        # Double lstm with dropout
        lstm_cell = rnn.MultiRNNCell([
            rnn.DropoutWrapper(rnn.LSTMCell(self.lat_size), output_keep_prob=(1 - self.dr)),
            rnn.DropoutWrapper(rnn.LSTMCell(self.lat_size), output_keep_prob=(1 - self.dr))
        ])
        lstm_outputs, _ = tf.nn.dynamic_rnn(lstm_cell, self.X, dtype=tf.float32)
        lstm_out = tf.transpose(lstm_outputs, (1, 0, 2))[-1]
        
        # Dense
        self.pred_labels = slim.fully_connected(lstm_out, self.N_values)
        
        # Loss
        self.loss = tf.losses.softmax_cross_entropy(self.y, self.pred_labels)


def get_pretty_tensor_model(max_len, N_values):
    return MyModel(max_len, N_values)

In [7]:
get_model = get_pretty_tensor_model

In [8]:
import numpy as np

''' Build a 2-layer LSTM from a training corpus '''


def build_model(corpus, val_indices, max_len, N_epochs=128):
    # number of different values or words in corpus
    N_values = len(set(corpus))

    # cut the corpus into semi-redundant sequences of max_len values
    step = 3
    sentences = []
    next_values = []
    for i in range(0, len(corpus) - max_len, step):
        sentences.append(corpus[i: i + max_len])
        next_values.append(corpus[i + max_len])
    print('nb sequences:', len(sentences))

    # transform data into binary matrices
    X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
    y = np.zeros((len(sentences), N_values), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, val in enumerate(sentence):
            X[i, t, val_indices[val]] = 1
        y[i, val_indices[next_values[i]]] = 1

    model = get_model(max_len, N_values)
    model.fit(X, y, batch_size=128, n_epochs=N_epochs)

    return model

In [9]:
# build model
model = build_model(corpus=corpus, val_indices=val_indices,
                         max_len=max_len, N_epochs=N_epochs)

# set up audio stream
out_stream = stream.Stream()

# generation loop
curr_offset = 0.0
loopEnd = len(chords)
for loopIndex in range(1, loopEnd):
    # get chords from file
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)

    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus,
                                      abstract_grammars=abstract_grammars,
                                      values=values, val_indices=val_indices,
                                      indices_val=indices_val,
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)

    curr_grammar = curr_grammar.replace(' A', ' C').replace(' X', ' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)

    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
                                            if isinstance(i, note.Note)])))

    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)

    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

# Play the final stream through output (see 'play' lambda function above)
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

# save stream
mf = midi.translate.streamToMidiFile(out_stream)
mf.open(out_fn, 'wb')
mf.write()
mf.close()    

nb sequences: 58


HBox(children=(IntProgress(value=0, description='Epoch', max=128), HTML(value='')))


After pruning: 10 notes
After pruning: 13 notes
After pruning: 11 notes
After pruning: 15 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 14 notes
After pruning: 14 notes
After pruning: 15 notes
After pruning: 13 notes
After pruning: 12 notes
After pruning: 14 notes
After pruning: 14 notes
After pruning: 15 notes
After pruning: 13 notes
After pruning: 13 notes
After pruning: 16 notes
After pruning: 11 notes


You can play generated sample using any midi player.

Under linux I prefer timidity.

In [10]:
! timidity midi/deepjazz_on_metheny...128

Check URL type=7
Check URL type=2
Check URL type=1
open url (type=1, name=/usr/local/Cellar/timidity/2.14.0/share/timidity/timidity.cfg)
url_file_open(/usr/local/Cellar/timidity/2.14.0/share/timidity/timidity.cfg)
mmap - success. size=4625
Check URL type=7
Check URL type=2
Check URL type=1
open url (type=1, name=~/.timidity.cfg)
url_file_open(~/.timidity.cfg)
Playing midi/deepjazz_on_metheny...128
MIDI file: midi/deepjazz_on_metheny...128
Check URL type=7
Check URL type=2
Check URL type=1
open url (type=1, name=midi/deepjazz_on_metheny...128)
url_file_open(midi/deepjazz_on_metheny...128)
mmap - success. size=3957
Format: 1  Tracks: 1  Divisions: 1024
Sequence: 
Check URL type=7
Check URL type=2
Check URL type=1
open url (type=1, name=Tone_000/000_Acoustic_Grand_Piano.pat)
url_file_open(Tone_000/000_Acoustic_Grand_Piano.pat)
Check URL type=7
Check URL type=2
Check URL type=1
open url (type=1, name=/usr/local/Cellar/timidity/2.14.0/share/timidity/Tone_000/000_Acoustic_Grand_Piano.pat)
ur